<a href="https://colab.research.google.com/github/karthikmohan1702/EIP_4/blob/master/v2___PersonAttrubutes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/EIP_4_Computer_vision/Week_5/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
replace resized/9733.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
gdrive/  hvc_annotations.csv  resized/  sample_data/


In [0]:
%tensorflow_version 1.x

import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from keras.applications import VGG16
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator


In [0]:
import numpy as np


def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
#def get_random_eraser(p=0.1, s_l=0.01, s_h=0.1, r_1=0.1, r_2=1/0.1, v_l=0, v_h=20, pixel_level=False):  
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

In [0]:
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Add, BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils

from keras.datasets import mnist

In [29]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

,gender,imagequality,age,weight,carryingbag,footwear,emotion,bodypose,image_path
0,male,Average,35-45,normal-healthy,Grocery/Home/Plastic Bag,Normal,Neutral,Front-Frontish,resized/1.jpg
1,female,Average,35-45,over-weight,None,Normal,Angry/Serious,Front-Frontish,resized/2.jpg
2,male,Good,45-55,normal-healthy,Grocery/Home/Plastic Bag,CantSee,Neutral,Front-Frontish,resized/3.jpg
3,male,Good,45-55,normal-healthy,Daily/Office/Work Bag,Normal,Neutral,Front-Frontish,resized/4.jpg
4,female,Good,35-45,slightly-overweight,None,CantSee,Neutral,Front-Frontish,resized/5.jpg


In [30]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

,0,1,2,3,4
image_path,resized/1.jpg,resized/2.jpg,resized/3.jpg,resized/4.jpg,resized/5.jpg
gender_female,0,1,0,0,1
gender_male,1,0,1,1,0
imagequality_Average,1,1,0,0,0
imagequality_Bad,0,0,0,0,0
imagequality_Good,0,0,1,1,1
age_15-25,0,0,0,0,0
age_25-35,0,0,0,0,0
age_35-45,1,1,0,0,1
age_45-55,0,0,1,1,0


In [0]:
import keras
import numpy as np

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, batch_size=32, shuffle=True, augmentation=None):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        self.augmentation = augmentation

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        image = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])
        if self.augmentation is not None:
          image = self.augmentation.flow(image,shuffle=False).next()
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)


In [32]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.15)
train_df.shape, val_df.shape

((11537, 28), (2036, 28))

In [33]:
train_df.head()

,image_path,gender_female,gender_male,imagequality_Average,imagequality_Bad,imagequality_Good,age_15-25,age_25-35,age_35-45,age_45-55,age_55+,weight_normal-healthy,weight_over-weight,weight_slightly-overweight,weight_underweight,carryingbag_Daily/Office/Work Bag,carryingbag_Grocery/Home/Plastic Bag,carryingbag_None,footwear_CantSee,footwear_Fancy,footwear_Normal,emotion_Angry/Serious,emotion_Happy,emotion_Neutral,emotion_Sad,bodypose_Back,bodypose_Front-Frontish,bodypose_Side
915,resized/916.jpg,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0
6382,resized/6383.jpg,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0
11661,resized/11663.jpg,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1
2194,resized/2195.jpg,1,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0
9824,resized/9825.jpg,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,1


In [0]:
# create train and validation data generators
train_gen = PersonDataGenerator(train_df, batch_size=32,augmentation = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rescale=1./255,preprocessing_function=get_random_eraser()) )
valid_gen = PersonDataGenerator(val_df, batch_size=32,augmentation = ImageDataGenerator(rescale=1./255))

In [35]:
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

{'age': 5,
 'bag': 3,
 'emotion': 4,
 'footwear': 3,
 'gender': 2,
 'image_quality': 3,
 'pose': 3,
 'weight': 4}

In [0]:
from tensorflow.keras.layers import Conv2D, SeparableConv2D, Activation, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Dropout, Flatten, Dense
"""backbone = VGG16(
    weights="imagenet", 
    include_top=False, 
    input_tensor=Input(shape=(224, 224, 3))
)"""
from keras.layers import Activation
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import AveragePooling2D, Input, Flatten,Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np

#model = Sequential()
inp = Input(shape = (224, 224, 3))
x = Convolution2D(32, (3, 3), padding = 'valid')(inp) #222
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2D(64, (3, 3))(x) #220
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = MaxPooling2D(pool_size = (2, 2))(x) #110
x = Dropout(0.2)(x)

x = Conv2D(128, (3, 3), padding = 'valid')(x) #108
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2D(64, (1, 1), padding = 'valid')(x)
x = Activation('relu')(x)

x = MaxPooling2D(pool_size = (2, 2))(x)#54
x = Dropout(0.25)(x)

x = Conv2D(128, (3, 3))(x)#52
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = MaxPooling2D(pool_size = (2, 2))(x)#26
x = Dropout(0.2)(x)

x = Conv2D(256, (3, 3))(x)#24
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = MaxPooling2D(pool_size = (2, 2))(x)#12
x = Dropout(0.2)(x)

x = Conv2D(512, (3, 3))(x)#10
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2D(128, (1, 1), padding = 'valid')(x)
x = Activation('relu')(x)

#x = MaxPooling2D(pool_size = (2, 2))(x)#8
#x = Dropout(0.2)(x)

#x = keras.layers.GlobalMaxPooling2D()(x)
x = AveragePooling2D(pool_size=8)(x)


#neck = backbone.output
#neck = Flatten(name="flatten")(neck)
#neck = Dense(512, activation="relu")(neck)


#def build_tower(in_layer):

    #neck = Dropout(0.2)(in_layer)
    #neck = Dense(128, activation="relu")(neck)
    #neck = Dropout(0.3)(in_layer)
    #neck = Dense(128, activation="relu")(neck)
 #   return neck


#def build_head(name, in_layer):
   # return 

x = Flatten(name="flatten")(x)
gender = Dense(num_units["gender"], activation="sigmoid", name=f"{'gender'}_output")(x)
image_quality = Dense(num_units["image_quality"], activation="softmax", name=f"{'image_quality'}_output")(x)
age = Dense(num_units["age"], activation="softmax", name=f"{'age'}_output")(x)
weight = Dense(num_units["weight"], activation="softmax", name=f"{'weight'}_output")(x)
bag = Dense(num_units["bag"], activation="softmax", name=f"{'bag'}_output")(x)
footwear = Dense(num_units["footwear"], activation="softmax", name=f"{'footwear'}_output")(x)
emotion = Dense(num_units["emotion"], activation="softmax", name=f"{'emotion'}_output")(x)
pose = Dense(num_units["pose"], activation="softmax", name=f"{'pose'}_output")(x)


# heads
"""gender = build_head("gender", build_tower(neck))
image_quality = build_head("image_quality", build_tower(neck))
age = build_head("age", build_tower(neck))
weight = build_head("weight", build_tower(neck))
bag = build_head("bag", build_tower(neck))
footwear = build_head("footwear", build_tower(neck))
emotion = build_head("emotion", build_tower(neck))
pose = build_head("pose", build_tower(neck))"""


model = Model(inputs=inp, outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion])

def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 70:
        lr *= 0.5e-3
    elif epoch > 60:
        lr *= 1e-3
    elif epoch > 50:
        lr *= 1e-2
    elif epoch > 40:
        lr *= 1e-1

        ###edited above learning rate
    """lr = 1e-3
    if epoch > 80:
        lr *= 0.5e-3
    elif epoch > 70:
        lr *= 1e-3
    elif epoch > 60:
        lr *= 1e-2
    elif epoch > 50:
        lr *= 1e-1"""



    """lr = 1e-3
    if epoch > 10:
        lr *= 1e-1
    elif epoch > 25:
        lr *= 1e-2
    elif epoch > 35:
        lr *= 1e-1"""

    
    """lr = 3e-3
    if epoch > 80:
        lr *= 0.5e-3
    elif epoch > 70:
        lr *= 1e-3
    elif epoch > 45:
        lr *= 1e-2
    elif epoch > 35:
        lr *= 1e-1"""


    print('Learning rate: ', lr)
    return lr

In [46]:
import os
os.getcwd()

'/content'

In [47]:
from keras.callbacks import ModelCheckpoint
save_dir = os.path.join(os.getcwd(), 'gdrive/My Drive/EIP_4_Computer_vision/Week_5/saved_models')
print(save_dir)
model_name = 'v2_person_attr%s_model.{epoch:03d}.h5' 
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True)

/content/gdrive/My Drive/EIP_4_Computer_vision/Week_5/saved_models


In [0]:
callbacks = [checkpoint]

In [49]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_17 (Conv2D)              (None, 222, 222, 32) 896         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_13 (BatchNo (None, 222, 222, 32) 128         conv2d_17[0][0]                  
__________________________________________________________________________________________________
activation_17 (Activation)      (None, 222, 222, 32) 0           batch_normalization_13[0][0]     
____________________________________________________________________________________________

In [0]:
#from keras.utils import plot_model
#plot_model(model)

In [51]:
"""# freeze backbone
for layer in backbone.layers:
	layer.trainable = False"""

'# freeze backbone\nfor layer in backbone.layers:\n\tlayer.trainable = False'

In [52]:
# losses = {
# 	"gender_output": "binary_crossentropy",
# 	"image_quality_output": "categorical_crossentropy",
# 	"age_output": "categorical_crossentropy",
# 	"weight_output": "categorical_crossentropy",

# }
# loss_weights = {"gender_output": 1.0, "image_quality_output": 1.0, "age_output": 1.0}
#opt = SGD(lr=0.001, momentum=0.9)
#model.compile(
 #   optimizer=opt,
  #  loss="categorical_crossentropy", 
    # loss_weights=loss_weights, 
   # metrics=["accuracy"]
#)

##########################
from keras.optimizers import Adam
losses = {
 	"gender_output": "binary_crossentropy",
 	"image_quality_output": "categorical_crossentropy",
 	"age_output": "categorical_crossentropy",
 	"weight_output": "categorical_crossentropy",
  "bag_output": "categorical_crossentropy",
  "pose_output": "categorical_crossentropy",
  "footwear_output": "categorical_crossentropy",
  "emotion_output": "categorical_crossentropy",
    }
"""loss_weights = {"gender_output": 1.0, "image_quality_output": 1.0, "age_output": 1.0,"weight_output":1.0,
                "bag_output":1.0,"pose_output":1.0,"footwear_output":1.0,"emotion_output":1.0 }"""
opt = SGD(lr=0.001, momentum=0.9)
#Adam(lr=lr_schedule(0))
model.compile(optimizer=Adam(lr=lr_schedule(0)),loss=losses, metrics=["accuracy"])

Learning rate:  0.001


In [0]:
#model.load_weights("weights.best.hdf5")
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
# model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=32, epochs=10)

In [55]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=2, 
    epochs=100,
    verbose=1,
    callbacks=callbacks
)

Epoch 1/100
360/360 [==============================] - 66s 182ms/step - loss: 7.8394 - gender_output_loss: 0.6815 - image_quality_output_loss: 0.9551 - age_output_loss: 1.4393 - weight_output_loss: 0.9893 - bag_output_loss: 0.9305 - footwear_output_loss: 0.9866 - pose_output_loss: 0.9392 - emotion_output_loss: 0.9180 - gender_output_acc: 0.5709 - image_quality_output_acc: 0.5464 - age_output_acc: 0.3947 - weight_output_acc: 0.6356 - bag_output_acc: 0.5529 - footwear_output_acc: 0.5306 - pose_output_acc: 0.6124 - emotion_output_acc: 0.7095 - val_loss: 7.7171 - val_gender_output_loss: 0.6713 - val_image_quality_output_loss: 0.9093 - val_age_output_loss: 1.4300 - val_weight_output_loss: 1.0004 - val_bag_output_loss: 0.9188 - val_footwear_output_loss: 0.9679 - val_pose_output_loss: 0.9193 - val_emotion_output_loss: 0.9001 - val_gender_output_acc: 0.5871 - val_image_quality_output_acc: 0.5486 - val_age_output_acc: 0.3924 - val_weight_output_acc: 0.6235 - val_bag_output_acc: 0.5625 - val_foo

In [56]:
res = model.evaluate_generator(valid_gen,verbose=1)
dict(zip(model.metrics_names,res))

63/63 [==============================] - 4s 71ms/step


{'age_output_acc': 0.38839285714285715,
 'age_output_loss': 1.3767806178047544,
 'bag_output_acc': 0.6398809523809523,
 'bag_output_loss': 0.9241989586088393,
 'emotion_output_acc': 0.6929563492063492,
 'emotion_output_loss': 0.8938961653482347,
 'footwear_output_acc': 0.6041666666666666,
 'footwear_output_loss': 0.9886386621566046,
 'gender_output_acc': 0.8353174603174603,
 'gender_output_loss': 0.4114305093174889,
 'image_quality_output_acc': 0.5684523809523809,
 'image_quality_output_loss': 0.912076921690078,
 'loss': 7.0638669861687555,
 'pose_output_acc': 0.816468253968254,
 'pose_output_loss': 0.5212062987543288,
 'weight_output_acc': 0.6026785714285714,
 'weight_output_loss': 1.0356388593476915}